https://datadoctorblog.com/2021/10/09/etc-stock-daily-min-chart-202110-w1/

페이지에서 다운로드 받은 데이터를 사용.  
파일은 압축파일명과 같은 "stock_kr_daily_m_211005_211008" 에 푸는 것을 권장

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [55]:
stock_nm = "BNK금융지주"

In [56]:
target_folder = "stock_kr_daily_m_211005_211008/"
list_files = os.listdir(target_folder)
list_files_m = [n_file for n_file in list_files if n_file[:7] == "stock_m"]
list_files_l = [n_file for n_file in list_files if n_file[:7] == "stock_l"]

In [57]:
df_list = pd.read_csv(target_folder + list_files_l[-1])
stock_cd = df_list.loc[df_list["stock_nm"] == stock_nm, "stock_cd"]
stock_cd = stock_cd.values[0]

In [67]:
df = pd.DataFrame()
for n_file in list_files_m:
    df_sub = pd.read_csv(target_folder + n_file, low_memory = False)
    df_sub = df_sub.loc[df_sub["stock_cd"] == stock_cd, ]
    df_sub["date"] = n_file.split("_")[2][:-4]
    df = pd.concat([df, df_sub])
    
df = df.reset_index(drop = True)

In [124]:
df["cont_time"] = df["cont_time"].astype("str").str.zfill(6)
df["datetime"] = pd.to_datetime(df["date"] + df["cont_time"])
ser_dates = pd.Series(df["date"].unique()).sort_values(ascending = False).values
df["datetime_same"] = str(ser_dates[0]) + df["cont_time"].astype("str")
df["datetime_same"] = pd.to_datetime(df["datetime_same"])

In [125]:
df.tail(2)

stock_cd cont_time  price  p_start  p_max  p_min  cum_cnt  cont_cnt  \
1502   138930    090100   8690     8670   8690   8660    50627      9294   
1503   138930    090000   8670     8730   8740   8650    41333     41333   

          date            datetime       datetime_same  end_date_width  
1502  20211008 2021-10-08 09:01:00 2021-10-08 09:01:00             1.2  
1503  20211008 2021-10-08 09:00:00 2021-10-08 09:00:00             1.2

In [129]:
df_last_day = df.loc[df["date"] == ser_dates[0], ].reset_index(drop = True)
point_start = df_last_day.iloc[[df_last_day["datetime"].idxmin()], ]
point_end   = df_last_day.iloc[[df_last_day["datetime"].idxmax()], ]

In [130]:
if point_start["price"].values[0] < point_end["price"].values[0]:
    plot_facecolor = "#FFAACC"
elif point_start["price"].values[0] > point_end["price"].values[0]:
    plot_facecolor = "#0C69F6"
else:
    plot_facecolor = "#999999"

In [132]:
df["end_date_width"] = (df["date"] == point_end["date"][0]) + 1.2

In [134]:
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.figure(figsize = (16, 6))
g = sns.lineplot(x = "datetime_same", y = "price", 
                 hue = "date", size = "end_date_width", data = df)
g.margins(x = 0.01, y = 0.1)
g.grid(which = "major", color = "#000000", alpha = 0.2, linewidth = 2)
g.legend(labels = df["date"].unique(),
         bbox_to_anchor = (1.02, 1), loc = "upper left", borderaxespad = 0)
g.set_facecolor(plot_facecolor)
g.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
g.set_title("{stock_nm}({stock_cd})".format(stock_nm = stock_nm, stock_cd = stock_cd), fontsize = 25)
g.set_xlabel(None)
g.set_ylabel("Price", fontsize = 20)
g.patch.set_alpha(0.2)
g.scatter(x = point_end["datetime_same"], y = point_end["price"], 
          c = "#FF0000", s = 80)
g;